# Running checklist test suite for SQuAD
source: code from https://github.com/marcotcr/checklist/blob/115f123de47ab015b2c3a6baebaffb40bab80c9f/notebooks/tutorials/5.%20Testing%20transformer%20pipelines.ipynb with minor changes
        

In [1]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
from checklist.test_suite import TestSuite
import numpy as np
import spacy
from checklist.perturb import Perturb
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
    AutoModelForQuestionAnswering, Trainer, TrainingArguments, HfArgumentParser
from transformers import pipeline 


In [12]:

model = pipeline('question-answering')
model({
    'context': 'A new strain of flu that has the potential to become a pandemic has been identified by scientists.',
    'question': 'What has been discovered by scientists?'
})

# to load your own model
# model_name = "trained_model_squad1/"
# model = AutoModelForQuestionAnswering.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# model = pipeline('question-answering', model=model, tokenizer=tokenizer)


No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


In [3]:
suite_path = 'squad_suite.pkl'
suite = TestSuite.from_file(suite_path)

In [4]:
def predconfs(context_question_pairs):
    preds = []
    confs = []
    for c, q in context_question_pairs:
        try:
            p = model(question=q, context=c, truncation=True, )
        except:
            print('Failed', q)
            preds.append(' ')
            confs.append(1)
        preds.append(p['answer'])
        confs.append(p['score'])
    return preds, np.array(confs)

In [5]:
suite.run(predconfs, n=100, overwrite=True)

Running A is COMP than B. Who is more COMP?
Predicting 100 examples


/opt/anaconda3/envs/nlp/lib/python3.9/site-packages/transformers/pipelines/question_answering.py:316: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  fw_args = {k: torch.tensor(v, device=self.device) for (k, v) in fw_args.items()}


Running A is COMP than B. Who is less COMP?
Predicting 100 examples
Running Intensifiers (very, super, extremely) and reducers (somewhat, kinda, etc)?
Predicting 1200 examples
Running size, shape, age, color
Predicting 400 examples
Running Profession vs nationality
Predicting 1000 examples
Running Animal vs Vehicle
Predicting 400 examples
Running Animal vs Vehicle v2
Predicting 400 examples
Running Synonyms
Predicting 400 examples
Running A is COMP than B. Who is antonym(COMP)? B
Predicting 400 examples
Running A is more X than B. Who is more antonym(X)? B. Who is less X? B. Who is more X? A. Who is less antonym(X)? A.
Predicting 1600 examples
Running Question typo
Predicting 200 examples
Running Question contractions
Predicting 202 examples
Running Add random sentence to context
Predicting 300 examples
Running Change name everywhere
Predicting 1100 examples
Running Change location everywhere
Predicting 1100 examples
Running There was a change in profession
Predicting 200 examples
Runn

In [6]:
def format_squad_with_context(x, pred, conf, label=None, *args, **kwargs):
    c, q = x
    ret = 'C: %s\nQ: %s\n' % (c, q)
    if label is not None:
        ret += 'A: %s\n' % label
    ret += 'P: %s\n' % pred
    return ret

In [7]:
suite.summary(format_example_fn=format_squad_with_context)

Vocabulary

A is COMP than B. Who is more COMP?
Test cases:      499
Test cases run:  100
Fails (rate):    0 (0.0%)


A is COMP than B. Who is less COMP?
Test cases:      497
Test cases run:  100
Fails (rate):    100 (100.0%)

Example fails:
C: Marie is smaller than Larry.
Q: Who is less small?
A: Larry
P: Marie

----
C: Jane is darker than Jason.
Q: Who is less dark?
A: Jason
P: Jane

----
C: Michelle is wiser than Daniel.
Q: Who is less wise?
A: Daniel
P: Michelle

----


Intensifiers (very, super, extremely) and reducers (somewhat, kinda, etc)?
Test cases:      498
Test cases run:  100
Fails (rate):    100 (100.0%)

Example fails:
C: Laura is pessimistic about the project. Amy is particularly pessimistic about the project.
Q: Who is least pessimistic about the project?
A: Laura
P: Amy

C: Amy is pessimistic about the project. Laura is somewhat pessimistic about the project.
Q: Who is most pessimistic about the project?
A: Amy
P: Laura

C: Laura is somewhat pessimistic about the proj

In [10]:
suite.visual_summary_table()

Please wait as we prepare the table data...


SuiteSummarizer(stats={'npassed': 0, 'nfailed': 0, 'nfiltered': 0}, test_infos=[{'name': 'A is COMP than B. Wh…